In [1]:
import inspect
import logging
logging.root.setLevel(logging.DEBUG)

import rpy2.robjects
import rpy2.robjects.pandas2ri
import rpy2.robjects.numpy2ri
import rpy2.robjects.conversion 
from rpy2.robjects.conversion import Converter


In [2]:
code = """
data(iris)
summary(lm(Sepal.Length ~ Petal.Width, iris))
"""
summaryA = rpy2.robjects.r(code)

In [3]:
low = Converter('low')
high = Converter('high')

from rpy2.robjects.vectors import ListVector, Vector
from rpy2.rinterface import Sexp, SexpVector
def Sexp_dict(robject):
    res = {}
    for i, (key, val) in enumerate(dict(robject.slots).items()):
        # list/dict type
        if key is None or isinstance(key, rpy2.rinterface.RNULLType) :
            key = i
        # try and update objects
        try:
            res[key] = rpy2.robjects.conversion.converter.ri2py(val)
        except ValueError:
            res[key] = val
    return res


def ListVector_dict(listvector):
    res = {}
    for i, (key, val) in enumerate(listvector.items()):
        logging.debug('serializing: %s', key)
        # list/dict type
        if key is None or isinstance(key, rpy2.rinterface.RNULLType) :
            key = i
        # try and update objects
        try:
            logging.debug("serializing %s with: %s", key, rpy2.robjects.conversion.converter.ri2py.dispatch(val.__class__))
            res[key] = rpy2.robjects.conversion.converter.ri2py(val)
        except ValueError:
            logging.exception('could not convert: %s', key)
            res[key] = val
    return res
def SexpVector_list(vector):
    # don't lookup names (buggy)
    res = []
    for val in vector:
        try:
            res.append(rpy2.robjects.conversion.converter.ri2py(val))
        except ValueError:
            res.append(val)
    return res

    

high.ri2py.register(SexpVector, SexpVector_list)
high.ri2py.register(Sexp, Sexp_dict)
high.ri2py.register(ListVector, ListVector_dict)


new_converter =  high + rpy2.robjects.pandas2ri.converter + rpy2.robjects.numpy2ri.converter + rpy2.robjects.conversion.converter
rpy2.robjects.conversion.set_conversion(new_converter)



In [4]:
code = """
data(iris)
fit <- lm(Sepal.Length ~ Petal.Width, iris)
summary(fit)
"""
summary = rpy2.robjects.r(code)


DEBUG:root:serializing: call
DEBUG:root:serializing call with: <function ri2py_vector at 0x1089590d0>
DEBUG:root:serializing: terms
DEBUG:root:serializing terms with: <function ri2py_sexp at 0x108956510>
DEBUG:root:serializing: residuals
DEBUG:root:serializing residuals with: <function ri2py_vector at 0x1089590d0>
DEBUG:root:serializing: coefficients
DEBUG:root:serializing coefficients with: <function ri2py_vector at 0x1089590d0>
DEBUG:root:serializing: aliased
DEBUG:root:serializing aliased with: <function ri2py_vector at 0x1089590d0>
DEBUG:root:serializing: sigma
DEBUG:root:serializing sigma with: <function ri2py_vector at 0x1089590d0>
DEBUG:root:serializing: df
DEBUG:root:serializing df with: <function ri2py_intvector at 0x108959158>
DEBUG:root:serializing: r.squared
DEBUG:root:serializing r.squared with: <function ri2py_vector at 0x1089590d0>
DEBUG:root:serializing: adj.r.squared
DEBUG:root:serializing adj.r.squared with: <function ri2py_vector at 0x1089590d0>
DEBUG:root:serializin

In [5]:
import json
import numpy as np
def encode_difficult(obj):
    if isinstance(obj, (np.ndarray,)):
        # if it's just a scalar
        if not np.squeeze(obj).ndim:
            return np.squeeze(obj).item()
        return obj.tolist()
    elif isinstance(obj, (rpy2.rinterface.Sexp, )):
        return {'robj': repr(obj)}
    raise TypeError("%r not serializable" % (obj, ))
json.dumps(summary, default=encode_difficult)

'{"fstatistic": [299.16731206935555, 1.0, 148.0], "sigma": 0.477994753562429, "adj.r.squared": 0.6667913866278083, "residuals": [0.1446545336371695, -0.055345466362838304, -0.25534546636284033, -0.35534546636284087, 0.044654533637159474, 0.266938482875887, -0.4442034917434773, 0.044654533637159474, -0.5553454663628401, 0.03351255901779612, 0.44465453363715984, -0.15534546636284072, -0.06648744098220441, -0.5664874409822044, 0.8446545336371594, 0.5669384828758869, 0.266938482875887, 0.055796508256522714, 0.6557965082565232, 0.055796508256522714, 0.44465453363715984, -0.03306151712411369, -0.35534546636284087, -0.12191954250474997, -0.15534546636284072, 0.044654533637159474, -0.13306151712411335, 0.24465453363715964, 0.24465453363715964, -0.25534546636284033, -0.15534546636284072, 0.266938482875887, 0.3335125590177959, 0.5446545336371595, -0.05534546636284018, 0.044654533637159474, 0.5446545336371595, 0.03351255901779612, -0.5553454663628401, 0.1446545336371591, -0.04420349174347693, -0.

In [6]:
np.squeeze(summary['adj.r.squared'])

array(0.6667913866278083)